In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 torchtext==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 841.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled t

In [ ]:
!pip install --upgrade spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [ ]:
import unicodedata
import string
import re
import random
import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torchtext
#from torchtext.legacy import data
import spacy
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:

import spacy.cli

spacy.cli.download("en_core_web_sm")
spacy.cli.download("fr_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [ ]:

import fr_core_news_sm
import en_core_web_sm

spacy_fr = fr_core_news_sm.load()
spacy_en = en_core_web_sm.load()

In [ ]:
### LATEST TORCHTEXT ###
from torchtext.data.utils import get_tokenizer

spacy_en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
spacy_fr_tokenizer = get_tokenizer("spacy", language="fr_core_news_sm")

In [ ]:
### LATEST TORCTHEXT ###

from collections import OrderedDict, Counter
from torchtext.vocab import Vocab
import io

path = './drive/My Drive/MachineTranslation/eng-fre/'
train_fn = 'train_eng_fre.tsv'
valid_fn = 'val_eng_fre.tsv'
test_fn = 'test_eng_fre.tsv'


def build_vocab(filepath, src_tokenizer, trg_tokenizer):
  src_counter, trg_counter = Counter(), Counter()
  with open(filepath, encoding="utf-8") as f:
    for i, line in enumerate(f.readlines()):
      if i == 0:  # skip header
        continue
      # split line and tokenize accordingly
      trg_line, src_line = line.strip("\n").split("\t")
      src_counter.update(src_tokenizer(src_line.lower()))
      trg_counter.update(trg_tokenizer(trg_line.lower()))
    
    # sort and wrap as OrderedDict
    # ordered_src = OrderedDict(sorted(src_counter.items(), key=lambda x: x[1], reverse=True))
    # ordered_trg = OrderedDict(sorted(trg_counter.items(), key=lambda x: x[1], reverse=True))
    ordered_src = sorted(src_counter.items(), key=lambda x: x[1], reverse=True)
    ordered_trg = sorted(trg_counter.items(), key=lambda x: x[1], reverse=True)
    
    # build vocab objects
    # NOTE: OrderedDict as input Requires torchtext >= 0.10.0. Using Counter for now
    src_vocab = Vocab(
      src_counter, 
      min_freq=2, 
      specials=('<unk>', '<pad>', '<bos>', '<eos>')
    )

    trg_vocab = Vocab(
      trg_counter, 
      min_freq=2,
      specials=('<unk>', '<pad>', '<bos>', '<eos>')
    )
    
    return src_vocab, trg_vocab

src_vocab, trg_vocab = build_vocab(
  path + train_fn, 
  spacy_fr_tokenizer,
  spacy_en_tokenizer
)

In [ ]:
### LATEST TORCHTEXT ###

import io

def data_process(path, split):
  raw_iter = iter(io.open(path + split, encoding="utf-8"))
  data = []
  for i, item in enumerate(raw_iter):
    if i == 0:
      continue
    trg_raw, src_raw = item.strip("\n").split("\t")
    src_tensor = torch.tensor(
        [src_vocab[token] for token in spacy_fr_tokenizer(src_raw.lower())],
        dtype=torch.long
      )
    trg_tensor = torch.tensor(
        [trg_vocab[token] for token in spacy_en_tokenizer(trg_raw.lower())],
        dtype=torch.long
      )
    data.append((src_tensor, trg_tensor))

  return data

In [ ]:
train_data = data_process(path, train_fn)
val_data = data_process(path, valid_fn)
test_data = data_process(path, test_fn)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(val_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [ ]:
trg_sent = [trg_vocab.itos[i] for i in train_data[0][1]]
src_sent = [src_vocab.itos[i] for i in train_data[0][0]]
print(trg_sent, src_sent)

['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.'] ['deux', 'jeunes', 'hommes', 'blancs', 'sont', 'dehors', 'près', 'de', 'buissons', '.']


In [ ]:
trg_sent = [trg_vocab.itos[i] for i in val_data[100][1]]
src_sent = [src_vocab.itos[i] for i in val_data[100][0]]
print(trg_sent, src_sent)

['an', 'older', ',', 'overweight', 'man', 'flips', 'a', 'pancake', 'while', 'making', 'breakfast', '.'] ['un', 'homme', 'âgé', 'en', 'surpoids', 'fait', 'sauter', 'une', 'crêpe', 'en', 'préparant', 'le', 'petit', 'déjeuner', '.']


In [ ]:
### LATEST TORCHTEXT ###

print(f"Unique tokens in source (fr) vocabulary: {len(src_vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(trg_vocab)}")

Unique tokens in source (fr) vocabulary: 6471
Unique tokens in target (en) vocabulary: 5893


In [ ]:
# trg_stoi = trg_vocab.get_stoi()  # torcthext >0.10.0
trg_stoi = trg_vocab.stoi
print(trg_stoi['<pad>'])

1


In [ ]:
### LATEST TORCHTEXT ###

BATCH_SIZE = {
    "train": 16,
    "val": 256,
    "test": 256
}

PAD_IDX = trg_vocab['<pad>']
BOS_IDX = trg_vocab['<bos>']
EOS_IDX = trg_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  src_batch, trg_batch = [], []
  for (src_item, trg_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
    trg_batch.append(torch.cat([torch.tensor([BOS_IDX]), trg_item, torch.tensor([EOS_IDX])], dim=0))
  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
  return src_batch, trg_batch

In [ ]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE["train"],
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE["val"],
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE["test"],
                       shuffle=True, collate_fn=generate_batch)

In [ ]:
# batch example of training data
for batch in train_iter:
    src, trg = batch
    print('tensor size of source language:', src.shape)
    print('tensor size of target language:', trg.shape)
    print('the tensor of first example in target language:', trg[:, 0])
    break

tensor size of source language: torch.Size([25, 16])
tensor size of target language: torch.Size([24, 16])
the tensor of first example in target language: tensor([  2,   7, 224, 106, 141,   7,  69,   5,   3,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
       
        # outputs are always from the top hidden layer, if bidirectional outputs are concatenated.
        # outputs shape [sequence_length, batch_size, hidden_dim * num_directions]
        # hidden is of shape [num_layers * num_directions, batch_size, hidden_size]
        # cell is of shape [num_layers * num_directions, batch_size, hidden_size]
        
        return outputs, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
             
        # input is of shape [batch_size]
        # hidden is of shape [n_layer * num_directions, batch_size, hidden_size]
        # cell is of shape [n_layer * num_directions, batch_size, hidden_size]
        
        input = input.unsqueeze(0)
        
        # input shape is [1, batch_size]. reshape is needed rnn expects a rank 3 tensors as input.
        # so reshaping to [1, batch_size] means a batch of batch_size each containing 1 index.
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]    
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        # output shape is [sequence_len, batch_size, hidden_dim * num_directions]
        # hidden shape is [num_layers * num_directions, batch_size, hidden_dim]
        # cell shape is [num_layers * num_directions, batch_size, hidden_dim]

        # sequence_len and num_directions will always be 1 in the decoder.
        # output shape is [1, batch_size, hidden_dim]
        # hidden shape is [num_layers, batch_size, hidden_dim]
        # cell shape is [num_layers, batch_size, hidden_dim]
        
        prediction = self.fc_out(hidden.squeeze(0)) # linear expects as rank 2 tensor as input
        # predicted shape is [batch_size, output_dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    ''' This class contains the implementation of complete sequence to sequence network.
    It uses to encoder to produce the context vectors.
    It uses the decoder to produce the predicted target sentence.
    Args:
        encoder: A Encoder class instance.
        decoder: A Decoder class instance.
    '''
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src is of shape [src_sequence_len, batch_size]
        # trg is of shape [targ_sequence_len, batch_size]
        # if teacher_forcing_ratio is 0.5 we use ground-truth inputs 50% of time and 50% time we use decoder outputs.

        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # to store the outputs of the decoder
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        # context vector, last hidden and cell state of encoder to initialize the decoder
        enc_outputs, cell = self.encoder(src)
        #print(enc_outputs.shape)
        hidden = torch.mean(enc_outputs, 0).unsqueeze(0)
        #print(hidden.shape)
        # here we dissect the cell tensor to get fwd and bwd passes
        fwd_cell = cell[0, :, :]
        bwd_cell = cell[1, :, :]
        # here we concatenate those two tensors to get
        # the cell tensor we need with shape [1, 16, 1024]
        cell = torch.cat((fwd_cell, bwd_cell), 1)
        cell = cell.unsqueeze(0)
        #print(cell.shape)

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            # pick a random number between 0 to ratio and decide whether to teacher force
            # if the ratio is 1.0, use_teacher_force is always 1 
            # if the ratio is 0.0, use_teacher_force is always 0
            # if the ration is 0.4, use_teacher_force is 1 for 40% of the time (on an average)
            use_teacher_force = random.random() < teacher_forcing_ratio 
            top1 = output.max(1)[1]
            # decide the next token based on use_teacher_force]
            # if teacher forcing is on, the next input will be the gold token from the previous timestep
            # otherwise, the next input will be the predicted token from the previous timestep.
            input = (trg[t] if use_teacher_force else top1) 

        # outputs is of shape [sequence_len, batch_size, output_dim]
        return outputs

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#INPUT_DIM = len(SRC.vocab) # tokens in source vocabulary
#OUTPUT_DIM = len(TRG.vocab) # tokens in target vocabulary
INPUT_DIM = len(src_vocab) # tokens in source vocabulary
OUTPUT_DIM = len(trg_vocab) # tokens in target vocabulary

#INPUT_DIM = 6004
#OUTPUT_DIM = 6004

# hyperparameters
ENC_EMB_DIM = 256 # encoder embedding size
DEC_EMB_DIM = 256 # decoder embedding size
ENC_HID_DIM = 512 # encoder hidden size
DEC_HID_DIM = 1024 # decoder hidden size
ENC_DROPOUT = 0.5 # dropout for encoder
DEC_DROPOUT = 0.3 # dropout for decoder
N_LAYERS = 1 # number of LSTM layers
LEARNING_RT = 0.001 # learning rate

# model
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6471, 256)
    (lstm): LSTM(256, 512, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (lstm): LSTM(256, 1024, dropout=0.3)
    (fc_out): Linear(in_features=1024, out_features=5893, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,610,501 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RT)

In [ ]:
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ', PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

<pad> token index:  1


In [ ]:
clip = 1
model.train()

for i, (src, trg) in enumerate(train_iter):
    
    # read the source sentence and target sentence
    # src = batch.SRC
    # trg = batch.TRG
    src, trg = src.to(device), trg.to(device)

    # clear the gradient buffer
    optimizer.zero_grad()

    # forward pass
    output = model(src, trg)
    #trg = [trg len, batch size]
    #output = [trg len, batch size, output dim]

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    #trg = [(trg len - 1) * batch size]
    #output = [(trg len - 1) * batch size, output dim]
    
    # compute the loss
    loss = criterion(output, trg)
    
    # compute the gradients
    loss.backward()

    # clip the gradients to prevent gradient explosion problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    
    # update the parameters
    optimizer.step()

    print(loss/src.shape[1])
    break

tensor(0.5426, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (src, trg) in enumerate(iterator):
        
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # loss function works only 2d logits, 1d targets
        # so flatten the trg, output tensors. Ignore the <sos> token
        # trg shape shape should be [(sequence_len - 1) * batch_size]
        # output shape should be [(sequence_len - 1) * batch_size, output_dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, (src, trg) in enumerate(iterator):

            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) # turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "./drive/My Drive/MachineTranslation/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 57s
	Train Loss: 4.301 | Train PPL:  73.763
	 Val. Loss: 4.517 |  Val. PPL:  91.584
Epoch: 02 | Time: 1m 57s
	Train Loss: 3.657 | Train PPL:  38.745
	 Val. Loss: 4.188 |  Val. PPL:  65.863
Epoch: 03 | Time: 1m 56s
	Train Loss: 3.245 | Train PPL:  25.649
	 Val. Loss: 3.929 |  Val. PPL:  50.844
Epoch: 04 | Time: 1m 56s
	Train Loss: 2.924 | Train PPL:  18.624
	 Val. Loss: 3.864 |  Val. PPL:  47.672
Epoch: 05 | Time: 1m 55s
	Train Loss: 2.656 | Train PPL:  14.237
	 Val. Loss: 3.736 |  Val. PPL:  41.939
Epoch: 06 | Time: 1m 55s
	Train Loss: 2.436 | Train PPL:  11.430
	 Val. Loss: 3.722 |  Val. PPL:  41.341
Epoch: 07 | Time: 1m 55s
	Train Loss: 2.220 | Train PPL:   9.207
	 Val. Loss: 3.800 |  Val. PPL:  44.696
Epoch: 08 | Time: 1m 55s
	Train Loss: 2.015 | Train PPL:   7.500
	 Val. Loss: 3.786 |  Val. PPL:  44.060
Epoch: 09 | Time: 1m 55s
	Train Loss: 1.829 | Train PPL:   6.228
	 Val. Loss: 3.878 |  Val. PPL:  48.310
Epoch: 10 | Time: 1m 55s
	Train Loss: 1.654 | Train PPL

In [ ]:

# INPUT_DIM = len(SRC_saved.vocab)
# OUTPUT_DIM = len(TRG_saved.vocab)
INPUT_DIM = len(src_vocab) # tokens in source vocabulary
OUTPUT_DIM = len(trg_vocab) # tokens in target vocabulary
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 1024
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.3
N_LAYERS = 1
LEARNING_RT = 0.001
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)

model_best = Seq2Seq(enc, dec, device)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
model_best.load_state_dict(torch.load('./drive/My Drive/MachineTranslation/seq2seq_6.pt')['state_dict'])
model_best = model_best.to(device)

In [ ]:
### LATEST TORCHTEXT ###

model_best.eval()
src_token = spacy_fr_tokenizer('Une femme avec un gros sac passe par une porte.')
print("src_token:", src_token)
src_tensor = torch.tensor([src_vocab[token.lower()] for token in src_token], dtype=torch.long)
src_tensor = torch.cat([torch.tensor([BOS_IDX]), src_tensor, torch.tensor([EOS_IDX])], dim=0)
src_tensor = src_tensor.unsqueeze(1).to(device)
print("shape of source language: ", src_tensor.shape)

src_token: ['Une', 'femme', 'avec', 'un', 'gros', 'sac', 'passe', 'par', 'une', 'porte', '.']
shape of source language:  torch.Size([13, 1])


In [ ]:
### LATEST TORCHTEXT ###
trg_token = ['<pad>']*64
trg_tensor = torch.tensor([trg_vocab[token.lower()] for token in trg_token], dtype=torch.long)
trg_tensor = torch.cat([torch.tensor([BOS_IDX]), trg_tensor, torch.tensor([EOS_IDX])], dim=0)
trg_tensor = trg_tensor.unsqueeze(1).to(device)
print("shape of target language: ", trg_tensor.shape)


shape of target language:  torch.Size([66, 1])


In [ ]:
output = model_best(src_tensor, trg_tensor, teacher_forcing_ratio = 0.0)
output_dim = output.shape[-1]
# get translation results, we ignore first token <sos> in both translation and target sentences. 
# output_translate = [(trg len - 1), batch, output dim] output dim is size of target vocabulary. 
output_translate = output[1:]
print("shape of output translate: ", output_translate.size())

shape of output translate:  torch.Size([65, 1, 5893])


In [ ]:
source_language_token_ids = src_tensor[:,0].cpu().numpy()
print("token indices in source language: ", source_language_token_ids)
translation_logit = output_translate[:,0,:].squeeze(1)
print("shape of logits for each prediction token in target language", translation_logit.size())

token indices in source language:  [  2   6  19  15   4 185 149 212  65   6 109   5   3]
shape of logits for each prediction token in target language torch.Size([65, 5893])


In [ ]:
# Choose top 1 word from decoder's output, we get the probability and index of the word
prob, token_id = translation_logit.data.topk(1)
print("shape of unnormalized logits corresponding to the top prediction for each prediction token = ", prob.size())
target_language_token_ids_along_with_pad = token_id.squeeze(1).cpu().numpy()
print("token indices in target language: ", target_language_token_ids_along_with_pad)

shape of unnormalized logits corresponding to the top prediction for each prediction token =  torch.Size([65, 1])
token indices in target language:  [   4   14   13    4   59  645   10   41   49    4 1024    5    3    3
    3    3    3    3    3    3    3    3    3    3    3    3    3    3
    3    3    3    3    3    3    3    3    3    3    3    3    3    3
    3    3    3    3    3    3    3    3    3    3    3    3    3    3
    3    3    3    3    3    3    3    3    3]


In [ ]:
# get source langauge in text
src_language_token_strs = []
for i in source_language_token_ids:
    #if i == SRC.vocab.stoi['<eos>']:
    if i == src_vocab.stoi['<eos>']:
        break
    else:
        #token = SRC.vocab.itos[i]
        token = src_vocab.itos[i]
        src_language_token_strs.append(token)
print("Source language:", src_language_token_strs)

# get machine translation in text
trans_language = []
for i in target_language_token_ids_along_with_pad:
    #if i == TRG.vocab.stoi['<eos>']:
    if i == trg_vocab.stoi['<eos>']:
        break
    else:
        #token = TRG.vocab.itos[i]
        token = trg_vocab.itos[i]
        trans_language.append(token)
print("Our model translation: ",  ' '.join(trans_language))
print("Gold translation: ", "a woman with a large purse is walking by a gate.")

Source language: ['<bos>', 'une', 'femme', 'avec', 'un', 'gros', 'sac', 'passe', 'par', 'une', 'porte', '.']
Our model translation:  a woman with a large purse is walking by a gate .
Gold translation:  a woman with a large purse is walking by a gate.


In [ ]:
def inference(model, file_name, src_vocab, trg_vocab, attention=False, max_trg_len=64):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    file_name: the directoy of test file that the first column is target reference, and the second column is source language;
    trg_vocab: Target torchtext Field
    attention: the model returns attention weights or not.
    max_trg_len: the maximal length of translation text (optinal), default = 64

    Output:
    Corpus BLEU score.
    '''
    from nltk.translate.bleu_score import corpus_bleu
    from nltk.translate.bleu_score import sentence_bleu
    from torchtext.legacy.data import TabularDataset
    from torchtext.legacy.data import Iterator

    # convert index to text string
    def convert_itos(convert_vocab, token_ids):
        list_string = []
        for i in token_ids:
            if i == convert_vocab.vocab.stoi['<eos>']:
                break
            else:
                token = convert_vocab.vocab.itos[i]
                list_string.append(token)
        return list_string

    test = TabularDataset(
      path=file_name, # the root directory where the data lies
      format='tsv',
      skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
      fields=[('TRG', trg_vocab), ('SRC', src_vocab)])

    test_iter = Iterator(
        dataset=test, # we pass in the datasets we want the iterator to draw data from
        sort=False,
        batch_size=128,
        sort_key=None,
        shuffle=False,
        sort_within_batch=False,
        device=device,
        train=False
    )
  
    model.eval()
    all_trg = []
    all_translated_trg = []

    TRG_PAD_IDX = trg_vocab.vocab.stoi[trg_vocab.pad_token]

    with torch.no_grad():
    
        for i, batch in enumerate(test_iter):

            src = batch.SRC
            #src = [src len, batch size]

            trg = batch.TRG
            #trg = [trg len, batch size]

            batch_size = trg.shape[1]

            # create a placeholder for traget language with shape of [max_trg_len, batch_size] where all the elements are the index of <pad>. Then send to device
            trg_placeholder = torch.Tensor(max_trg_len, batch_size)
            trg_placeholder.fill_(TRG_PAD_IDX)
            trg_placeholder = trg_placeholder.long().to(device)
            if attention == True:
                output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
            else:
                #original 
                #output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
                
                # update:
                output = model(src, trg_placeholder, 0) #turn off teacher forcing
            # get translation results, we ignor first token <sos> in both translation and target sentences. 
            # output_translate = [(trg len - 1), batch, output dim] output dim is size of target vocabulary.
            output_translate = output[1:]
            # store gold target sentences to a list 
            all_trg.append(trg[1:].cpu())

            # Choose top 1 word from decoder's output, we get the probability and index of the word
            prob, token_id = output_translate.data.topk(1)
            translation_token_id = token_id.squeeze(2).cpu()

            # store gold target sentences to a list 
            all_translated_trg.append(translation_token_id)
      
    all_gold_text = []
    all_translated_text = []
    for i in range(len(all_trg)): 
        cur_gold = all_trg[i]
        cur_translation = all_translated_trg[i]
        for j in range(cur_gold.shape[1]):
            gold_convered_strings = convert_itos(trg_vocab,cur_gold[:, j])
            trans_convered_strings = convert_itos(trg_vocab,cur_translation[:, j])

            all_gold_text.append(gold_convered_strings)
            all_translated_text.append(trans_convered_strings)

    corpus_all_gold_text = [[item] for item in all_gold_text]
    corpus_bleu_score = corpus_bleu(corpus_all_gold_text, all_translated_text)  
    return corpus_bleu_score

In [ ]:
def data_process(path):
  raw_iter = iter(io.open(path, encoding="utf-8"))
  data = []
  for i, item in enumerate(raw_iter):
    if i == 0:
      continue
    trg_raw, src_raw = item.strip("\n").split("\t")
    src_tensor = torch.tensor(
        [src_vocab[token] for token in spacy_fr_tokenizer(src_raw.lower())],
        dtype=torch.long
      )
    trg_tensor = torch.tensor(
        [trg_vocab[token] for token in spacy_en_tokenizer(trg_raw.lower())],
        dtype=torch.long
      )
    data.append((src_tensor, trg_tensor))

  return data

def inference(model, file_name, src_vocab, trg_vocab, attention=False, max_trg_len=64):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    file_name: the directoy of test file that the first column is target reference, and the second column is source language;
    trg_vocab: Target torchtext Field
    attention: the model returns attention weights or not.
    max_trg_len: the maximal length of translation text (optinal), default = 64

    Output:
    Corpus BLEU score.
    '''
    from nltk.translate.bleu_score import corpus_bleu
    from nltk.translate.bleu_score import sentence_bleu

    # convert index to text string
    def convert_itos(convert_vocab, token_ids):
        list_string = []
        for i in token_ids:
            if i == convert_vocab.stoi['<eos>']:
                break
            else:
                token = convert_vocab.itos[i]
                list_string.append(token)
        return list_string
    test_data = data_process(file_name)
    test_iter = DataLoader(test_data, batch_size=BATCH_SIZE["test"],
                       shuffle=True, collate_fn=generate_batch)
  
    model.eval()
    all_trg = []
    all_translated_trg = []

    #TRG_PAD_IDX = trg_vocab.stoi[trg_vocab.pad_token]

    with torch.no_grad():
    
        for i, (src, trg) in enumerate(test_iter):

            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) # turn off teacher forcing


            output_translate = output[1:]
            translation_logit = output_translate[:,0,:].squeeze(1)
            # store gold target sentences to a list 
            all_trg.append(trg[1:].cpu())

            # Choose top 1 word from decoder's output, we get the probability and index of the word
            prob, token_id = output_translate.data.topk(1)
            translation_token_id = token_id.squeeze(2).cpu()

            target_language_token_ids_along_with_pad = token_id.squeeze(1).cpu().numpy()
            # store gold target sentences to a list 
            all_translated_trg.append(translation_token_id)
      
    all_gold_text = []
    all_translated_text = []
    #print(all_trg[0].shape)
    #print(all_translated_trg[0].shape)
    for i in range(len(all_trg)): 
        cur_gold = all_trg[i]
        cur_translation = all_translated_trg[i]
        #print(cur_translation)
        for j in range(cur_gold.shape[1]):
            gold_convered_strings = convert_itos(trg_vocab,cur_gold[:, j])
            #print(gold_convered_strings)
            
            trans_convered_strings = convert_itos(trg_vocab,cur_translation[:,j])
            #print(trans_convered_strings)
            #print()
            #print('xxxxxxx---------------------xxxxx')
            #print()
            all_gold_text.append(gold_convered_strings)
            all_translated_text.append(trans_convered_strings)

    corpus_all_gold_text = [[item] for item in all_gold_text]
    corpus_bleu_score = corpus_bleu(corpus_all_gold_text, all_translated_text)  
    return corpus_bleu_score

In [ ]:
inference(model_best, "./drive/My Drive/MachineTranslation/eng-fre/test_eng_fre.tsv", src_vocab, trg_vocab, True, 64)

0.2081102059159077